label encoded geography

Data Preporcessing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv(r"C:\Users\okechukwu chude\Documents\Neural Network\Customer Churn\Churn_Modelling.csv")
x = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
print(dataset.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

Encoding categorical data


In [4]:
# Label Encoding the "geography" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 1] = le.fit_transform(x[:, 1])
print(x)

[[619 0 'Female' ... 1 1 101348.88]
 [608 2 'Female' ... 0 1 112542.58]
 [502 0 'Female' ... 1 0 113931.57]
 ...
 [709 0 'Female' ... 0 1 42085.58]
 [772 1 'Male' ... 1 0 92888.52]
 [792 0 'Female' ... 1 0 38190.78]]


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])
print(x)

[[619 0 0 ... 1 1 101348.88]
 [608 2 0 ... 0 1 112542.58]
 [502 0 0 ... 1 0 113931.57]
 ...
 [709 0 0 ... 0 1 42085.58]
 [772 1 1 ... 1 0 92888.52]
 [792 0 0 ... 1 0 38190.78]]


In [6]:
# Get the first two rows
first_two_rows = x[:2]

print(first_two_rows)

[[619 0 0 42 2 0.0 1 1 1 101348.88]
 [608 2 0 41 1 83807.86 1 0 1 112542.58]]


Splitting into training and test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

Feature Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Developing the ANN

In [9]:
from scikeras.wrappers import KerasClassifier

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
from keras.models import Sequential #required to initialise ann
from keras.layers import Dense # required to build laters of ann

In [12]:
#function to build ANN architecture
def build_classifier():
    #initialise
    classifier = Sequential()
    #add input layer and first hidden layer
    classifier.add(Dense(units=6, activation='relu', input_dim=10))
    #add second hidden layer
    classifier.add(Dense(units=6, activation='relu'))
    #add output layer
    classifier.add(Dense(units=1, activation='sigmoid'))
    #compile ANN
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
    return classifier


In [13]:
#create global classifier variable
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [14]:
mean = accuracies.mean()
variance = accuracies.std()
print('mean accuracy:', mean , 'accuracy variance:', variance)

mean accuracy: 0.8568749999999999 accuracy variance: 0.012553012586626363


Handling Overfitting

In [15]:
from keras.layers import Dropout #to prevent overfitting

In [24]:
classifier = Sequential()

#add input layer and first hidden layer
classifier.add(Dense(units=6, activation='relu', input_dim=10))
classifier.add(Dropout(rate = 0.1))

#add second hidden layer
classifier.add(Dense(units=6, activation='relu'))
classifier.add(Dropout(rate = 0.1))

#add output layer
classifier.add(Dense(units=1, activation='sigmoid'))

#compile ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

TypeError: Dropout.__init__() missing 1 required positional argument: 'rate'

Tuning the ANN using GridSearch

In [17]:
from sklearn.model_selection import GridSearchCV

In [33]:
from keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

In [40]:
#function to build ANN architecture
def build_classifier(optimizer='adam'):
    #initialise
    classifier = Sequential()
    
    #add input layer and first hidden layer
    classifier.add(Dense(units=6, activation='relu', input_dim=10))
    
    #add second hidden layer
    classifier.add(Dense(units=6, activation='relu'))
    
    #add output layer
    classifier.add(Dense(units=1, activation='sigmoid'))
    
    #compile ANN
    classifier.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])
    return classifier

In [41]:
#create global classifier variable
classifier = KerasClassifier(build_fn = build_classifier)

Create Dictionary that contains the Hyper parameters

In [42]:
parameters = {'optimizer': ['adam', 'rmsprop'],
            'batch_size' :[25, 32],
             'epochs': [100, 500]
             }

In [43]:
grid_search = GridSearchCV(estimator= classifier,
                           param_grid= parameters,
                           scoring= 'accuracy',
                           cv = 10)

In [44]:
#fit gridseasrch to training set
grid_search = grid_search.fit(X_train, y_train)

C:\Users\okechukwu chude\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
288/288 [==============================] - 1s 2ms/step - loss: 0.5503 - accuracy: 0.7600
Epoch 2/100
288/288 [==============================] - 0s 2ms/step - loss: 0.4603 - accuracy: 0.7981
Epoch 3/100
288/288 [==============================] - 0s 2ms/step - loss: 0.4431 - accuracy: 0.8017
Epoch 4/100
288/288 [==============================] - 0s 2ms/step - loss: 0.4370 - accuracy: 0.8036
Epoch 5/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4333 - accuracy: 0.8089
Epoch 6/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4300 - accuracy: 0.8112
Epoch 7/100
288/288 [==============================] - 0s 2ms/step - loss: 0.4264 - accuracy: 0.8147
Epoch 8/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4213 - accuracy: 0.8188
Epoch 9/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4151 - accuracy: 0.8214
Epoch 10/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4085 - accura

C:\Users\okechukwu chude\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


288/288 [==============================] - 2s 3ms/step - loss: 0.6387 - accuracy: 0.6779
Epoch 2/100
288/288 [==============================] - 1s 2ms/step - loss: 0.5198 - accuracy: 0.7974
Epoch 3/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4640 - accuracy: 0.7994
Epoch 4/100
288/288 [==============================] - 1s 3ms/step - loss: 0.4382 - accuracy: 0.8046
Epoch 5/100
288/288 [==============================] - 1s 3ms/step - loss: 0.4239 - accuracy: 0.8140
Epoch 6/100
288/288 [==============================] - 1s 3ms/step - loss: 0.4111 - accuracy: 0.8304
Epoch 7/100
288/288 [==============================] - 1s 3ms/step - loss: 0.3983 - accuracy: 0.8397
Epoch 8/100
288/288 [==============================] - 1s 2ms/step - loss: 0.3860 - accuracy: 0.8467
Epoch 9/100
288/288 [==============================] - 1s 3ms/step - loss: 0.3760 - accuracy: 0.8494
Epoch 10/100
288/288 [==============================] - 1s 3ms/step - loss: 0.3687 - accuracy: 0.8526
E

C:\Users\okechukwu chude\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

225/225 [==============================] - 1s 2ms/step - loss: 0.5579 - accuracy: 0.7874
Epoch 2/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4803 - accuracy: 0.7982
Epoch 3/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4473 - accuracy: 0.8093
Epoch 4/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8172
Epoch 5/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4237 - accuracy: 0.8215
Epoch 6/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4160 - accuracy: 0.8249
Epoch 7/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4086 - accuracy: 0.8283
Epoch 8/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4015 - accuracy: 0.8322
Epoch 9/500
225/225 [==============================] - 1s 2ms/step - loss: 0.3934 - accuracy: 0.8369
Epoch 10/500
225/225 [==============================] - 0s 2ms/step - loss: 0.3861 - accuracy: 0.8411
E

C:\Users\okechukwu chude\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


225/225 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7896
Epoch 2/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4852 - accuracy: 0.7961
Epoch 3/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4605 - accuracy: 0.7961
Epoch 4/500
225/225 [==============================] - 1s 2ms/step - loss: 0.4452 - accuracy: 0.7961
Epoch 5/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4369 - accuracy: 0.7961
Epoch 6/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4321 - accuracy: 0.8004
Epoch 7/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4294 - accuracy: 0.8085
Epoch 8/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4267 - accuracy: 0.8126
Epoch 9/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4233 - accuracy: 0.8165
Epoch 10/500
225/225 [==============================] - 0s 2ms/step - loss: 0.4199 - accuracy: 0.8222
E

C:\Users\okechukwu chude\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


250/250 [==============================] - 2s 2ms/step - loss: 0.6003 - accuracy: 0.6976
Epoch 2/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4785 - accuracy: 0.7961
Epoch 3/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4560 - accuracy: 0.7966
Epoch 4/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4418 - accuracy: 0.7997
Epoch 5/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4301 - accuracy: 0.8050
Epoch 6/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4176 - accuracy: 0.8115
Epoch 7/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4043 - accuracy: 0.8152
Epoch 8/500
250/250 [==============================] - 1s 2ms/step - loss: 0.3938 - accuracy: 0.8181
Epoch 9/500
250/250 [==============================] - 1s 2ms/step - loss: 0.3860 - accuracy: 0.8194
Epoch 10/500
250/250 [==============================] - 1s 2ms/step - loss: 0.3799 - accuracy: 0.8204
E

In [45]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [46]:
best_parameters

{'batch_size': 32, 'epochs': 500, 'optimizer': 'rmsprop'}

In [47]:
best_accuracy

0.8606250000000001